# Leonardo Favero Bocardi - RA: 120767

## Exercício

(1) Avaliar a combinação das técnicas de seleção de atributos e redução de dimensionalidade com as métricas de desempenho apropriadas e verificar se os comitês de máquina (ensemble) auxiliam na melhora do resultado.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score

import time

In [2]:
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target

---

## Hipótese 01 - Proposta em aula:
#### *Regressão Logística com validação cruzada é mais precisa em termos de acurácia quando comparada à Regressão Logística passando por Bagging Classifier de 20 estimators em HoldOut? E com Validação Cruzada?*

### Método 01: Regressão Logística c/ Validação Cruzada

In [3]:
start = time.time()
end = time.time()

model = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=2000)

scores = cross_val_score(model, X, y, cv=5)
print('Acurácia: %.4f +- %.4f' % (scores.mean() * 100, scores.std() * 100))
print('run time:',end - start)

Acurácia: 91.6000 +- 2.4363
run time: 3.3855438232421875e-05


### Método 02: Regressão Logística em Bagging Classifier de 20 estimators c/ HoldOut

In [4]:
start = time.time()
end = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = BaggingClassifier(LogisticRegression(), max_samples=0.5, max_features=0.5, n_estimators=20)
model.fit(X_train,y_train)

pred = model.predict(X_test)
print('Acurácia: %.4f' % (accuracy_score(y_test, pred)*100))
print('run time:',end - start)

Acurácia: 96.6667
run time: 4.291534423828125e-05


### Método 03: Regressão logística em Bagging Classifier de 20 Estimators c/ Validação Cruzada

In [5]:
start = time.time()
end = time.time()

model = BaggingClassifier(LogisticRegression(), max_samples=0.5, max_features=0.5, n_estimators=20)
model.fit(X_train,y_train)


scores = cross_val_score(model, X, y, cv=5)
print('Acurácia: %.4f +- %.4f' % (scores.mean() * 100, scores.std() * 100))
print('run time:',end - start)

Acurácia: 92.2098 +- 2.9133
run time: 3.409385681152344e-05


**Conclusão:** O método de Bagging Classifier com 20 estimators obteve um desempenho melhor das 2 maneiras quando comparados com o método simples somente com validação cruzada.<br>
Entre os métodos de Bagging Classifier, o método 02 obteve um resultado melhor em termos de acurácia, com um tempo de execução mais longo.<br>
O método 03 não obteve a mesma acurácia, porém foi executado mais rapidamente.

---

## Hipótese 02 - Random Forests:
#### *O Ensemble de Random Forest com Cross Validation é menos preciso e mais demorado do que o mesmo método após aplicar o PCA para seleção de atributos com 95% de variância*

### Método 01: Ensemble Random Forest com Cross Validation

In [18]:
start = time.time()
end = time.time()

model = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=2, random_state=0)

scores = cross_val_score(model, X, y, cv=5)
print('Acurácia de Random Forest: %.4f' % (scores.mean()*100))
print('run time:',end - start)

Acurácia de Random Forest: 93.2713
run time: 6.508827209472656e-05


### Método 02: Ensemble Random Forest com Cross Validation pós PCA de variância de 95%

In [19]:
start = time.time()
end = time.time()

pca = PCA(n_components=0.95, whiten=True)

X_pca = pca.fit_transform(X)

print('Número original de atributos:', X.shape[1])
print('Número reduzido de atributos:', X_pca.shape[1])
print('')

model = RandomForestClassifier(n_estimators=50, max_depth=None, min_samples_split=2, random_state=0)

scores = cross_val_score(model, X_pca, y, cv=5)
print('Acurácia de Random Forest: %.4f' % (scores.mean()*100))
print('run time:',end - start)

Número original de atributos: 64
Número reduzido de atributos: 29

Acurácia de Random Forest: 93.7755
run time: 3.5762786865234375e-05


**Conclusão:** 
Em termos de acurácia, ambos os métodos obtiveram o mesmo desempenho. Uma vez que a variância proposta para o método com PCA foi relativamente alta.<br>
Porém o método 02 obteve um desempenho superior em termos de tempo de execução, uma vez que foi analisada uma quantidade menor de atributos, mantendo os 95% de variância.